In [6]:
import Pkg
Pkg.add("Tullio")

   Resolving package versions...


   Installed DiffRules ─ v1.11.0


   Installed Tullio ──── v0.3.4


    Updating `~/.julia/environments/v1.6/Project.toml`
  [bc48ee85] 

+ Tullio v0.3.4
    Updating `~/.julia/environments/v1.6/Manifest.toml`
  [b552c78f] + DiffRules v1.11.0
  [bc48ee85] + Tullio v0.3.4


Precompiling 

project...


  ✓ DiffRules


  ✓ Tullio


  2 dependencies successfully precompiled in 11 seconds (157 already precompiled)


# Tullio.jl

El paquete `Tullio` permite escribir de manera sencilla y legible operaciones con arreglos sin tener que usar más de una línea o escribir varios ciclos for anidados.

Para utilizarlo solo es necesario agregar la biblioteca al espacio de trabajo y utilizar la macro `@tullio` seguido de la sentencia a ejecutar.

Esto permite escribir **sumas de Einstein** en programas de julia. El macro escribe ciclos anidados si se usa de la manera más sencilla. Sin embargo, es capaz de parsear muchas expresiones más e inferir los rangos de los índices.

Otra característica importante es que utiliza **hilos múltiples** (usando `Threads.@spawn`) y _tiling_ recursivo en arreglos suficientemente grandes. También puede trabajar en conjunto con otros paquetes en caso de encontrarse cargados antes de que se llame a la macro:

- Utiliza `LoopVectorization.@avx` para acelerar algunas cosas. Puede llegar a igualar el rendimiento de `OpenBLAS` en multiplicación de matrices.
- Utiliza `KernelAbstractions.@kernel` (y CUDAKernels) para crear una versión ejecutable en GPU. Esta función es experimental y puede no siempre ser rápida.

La sintaxis es sencilla. Todo lo que se encuentra a la derecha del igual (= o :=) se suma en todos los posibles rangos de cualquier índice que no aparezca a la izquierda.

Operadores de Pipe `|>` o `<|` indican funciones que se deben realizar fuera de la suma.

A continuación mostraremos algunos ejemplos para entender mejor cómo funciona este paquete.

In [9]:
using Tullio, Test

In [8]:
M = rand(1:20, 3,7)

3×7 Matrix{Int64}:
 14  12  12   2  20  18  13
  1   8  14  17  19   4   3
 12   6  20  18  15   7  10

In [10]:
#Se suma de r ϵ 1:3 , por cada c ϵ 1:7
@tullio S[1,c] := M[r,c]

1×7 Matrix{Int64}:
 27  26  46  37  54  29  26

El código de arriba sería equivalente al siguiente ciclo anidado. Cabe destacar que el símbolo `:=` le indica a Tullio que se creará un nuevo arreglo a partir del resultado de la opración. En caso de haber `=` solamente, se escribe el resultado sobre un arreglo ya existente.

In [14]:
S2 = zeros(1,7)
for c = 1:7
    for r =1:3
        S2[1,c] += M[r,c]
    end
end

@test S == S2

Test Passed

De la misma manera, hay funciones que simplifican la operación anterior. Sin embargo, Tullio puede hacer muchas cosas más complejas, con más rangos, que terminarían siendo muy enredadas e ilegibles si se escribieran con solamente ciclos anidados o usando el broadcasting integrado de Julia.

In [17]:
@tullio Q[p,c] := M[p,c] + sqrt(S[1,c])
@test Q ≈ M .+ sqrt.(S)

Test Passed

In [19]:
mult(M,Q) = @tullio P[x,y] := M[x,c] * Q[y,c]
@test mult(M,Q) ≈ M * transpose(Q)

Test Passed

## Operaciones avanzadas con índices

Además de solo escribir los índices y dejar que Tullio los infiera, es posible realizar muchas más operaciones variando los índices para obtener resultados más complicados. 

Por ejemplo, el intercambio de índices o su escalamiento también está permitido.

In [20]:
R = [rand(Int8, 3, 4) for d in 1:5]


5-element Vector{Matrix{Int8}}:
 [3 -47 108 -17; 53 126 6 -73; 73 21 49 -85]
 [3 -120 -41 -108; -128 -18 105 -74; -53 -36 113 -83]
 [60 70 69 57; -48 -3 -107 -38; 23 66 -45 44]
 [-13 60 69 100; -63 -125 -28 -5; -97 -101 44 -64]
 [80 121 41 12; 77 -28 -105 96; 113 -36 -49 86]

In [24]:
@tullio T[j,i,δ] := R[δ][i,j] + 10im 
@test T == permutedims(cat(R...; dims=3),(2,1,3)) .+ 10im

Test Passed

La expresión anterior reescribe una matriz cambiando las dimensiones y suma el número 10i a cada elemento. Como se puede apreciar en el test, es mucho más legible la sentencia escrita con Tullio que si quisiéramos hacerlo con funciones nativas de Julia.

Si quisieras utilizar una variable como escalar , es necesario escribirla con un signo de dolar, como `$a` o `A[i, $Y]`.

In [29]:
a = 2;
@tullio S3[1,c] := M[$a,c] + $a

1×7 Matrix{Int64}:
 3  10  16  19  21  6  5

De la misma manera, se pueden indicar los rangos por los que debe iterar tullio para que éste no los infiera. Los rangos se indican al final de la sentencia entre paréntesis.

In [44]:
A = [abs(i-11) for i in 1:21]

@tullio M[i,j] := A[i+j-1] (j in 1:15) # i en 1:7
@tullio M[i+_,j] := A[i+j] (j in 1:15) # i en 0:6 con shift automático "i+_ 

7×15 Matrix{Int64}:
 10  9  8  7  6  5  4  3  2  1  0  1  2  3   4
  9  8  7  6  5  4  3  2  1  0  1  2  3  4   5
  8  7  6  5  4  3  2  1  0  1  2  3  4  5   6
  7  6  5  4  3  2  1  0  1  2  3  4  5  6   7
  6  5  4  3  2  1  0  1  2  3  4  5  6  7   8
  5  4  3  2  1  0  1  2  3  4  5  6  7  8   9
  4  3  2  1  0  1  2  3  4  5  6  7  8  9  10

Además de realizar sumas, Tullio también es capaz de realizar otro tipo de operaciones. Por ejemplo, puede realizar **multiplicaciones** en lugar de sumas si agregas `(*)` después del macro `@tullio`.

In [48]:
C = rand(1:20, 3,7)
@tullio (*) E[1,i] := C[j,i]

1×7 Matrix{Int64}:
 36  1890  240  1120  728  304  612

De igual manera, puedes usar cualquier función de reducción para ejecutar otro tipo de operación. Por ejemplo:

In [25]:
@tullio (max) X[i] := abs(T[j,i,δ])

3-element Vector{Float64}:
 121.41251994749142
 128.39003076563228
 113.44161493913951

## Integración con OffsetArrays.jl

Tullio puede soportar operaciones en conjunto con el paquete `OffsetArrays.jl`, el cual nos permite trabajar con arreglos cuyos índices son arbitrarios y definidos por el usuario, función que se encuentra en lenguajes como Fortran.

Si quisiéramos utilizar un `OffsetArray` con Tullio, éste puede inferir los índices de la misma manera en que lo haría con un arreglo normal.

In [52]:
using OffsetArrays

K = OffsetArray([1,-1,2,-1,1], -2:2)

5-element OffsetArray(::Vector{Int64}, -2:2) with eltype Int64 with indices -2:2:
  1
 -1
  2
 -1
  1

In [57]:
@tullio C[i] := A[i+j] * K[j] # j ϵ -2:2 implica que i ϵ 3:19


17-element OffsetArray(::Vector{Int64}, 3:19) with eltype Int64 with indices 3:19:
 16
 14
 12
 10
  8
  6
  4
  4
  2
  4
  4
  6
  8
 10
 12
 14
 16

In [58]:
@tullio D[i,j] := A[2K[j] + i] ÷ K[j] #extrema(K) == (-1,2) implica i ϵ 3:17

15×5 OffsetArray(::Matrix{Int64}, 3:17, -2:2) with eltype Int64 with indices 3:17×-2:2:
 6  -10  2  -10  6
 5   -9  1   -9  5
 4   -8  1   -8  4
 3   -7  0   -7  3
 2   -6  0   -6  2
 1   -5  0   -5  1
 0   -4  1   -4  0
 1   -3  1   -3  1
 2   -2  2   -2  2
 3   -1  2   -1  3
 4    0  3    0  4
 5   -1  3   -1  5
 6   -2  4   -2  6
 7   -3  4   -3  7
 8   -4  5   -4  8

Incluso se puede usar la inferencia de índices multidimensionales y para acceder a miembros o arreglos en una tupla. Por ejemplo:

In [61]:
# Este ejemplo utiliza j ϵ eachindex(first(N).c)
N = [(a=i, b=i^2, c=fill(i^3,3)) for i in 1:10]
@tullio T[i,j] := (N[i].a // 1, N[i].c[j])

10×3 Matrix{Tuple{Rational{Int64}, Int64}}:
 (1//1, 1)      (1//1, 1)      (1//1, 1)
 (2//1, 8)      (2//1, 8)      (2//1, 8)
 (3//1, 27)     (3//1, 27)     (3//1, 27)
 (4//1, 64)     (4//1, 64)     (4//1, 64)
 (5//1, 125)    (5//1, 125)    (5//1, 125)
 (6//1, 216)    (6//1, 216)    (6//1, 216)
 (7//1, 343)    (7//1, 343)    (7//1, 343)
 (8//1, 512)    (8//1, 512)    (8//1, 512)
 (9//1, 729)    (9//1, 729)    (9//1, 729)
 (10//1, 1000)  (10//1, 1000)  (10//1, 1000)

## Expresiones más grandes

Aunque la simplicidad de Tullio recae en que queda todo señalado en una sola línea, no tiene que ser necesariamente siempre así. Por ejemplo:
@tullio out[x,y] := @inbounds(begin
    
)

In [63]:
using Tullio, OffsetArrays

mat = zeros(10,10,1); mat[2,2] = 101; mat[10,10] = 1;

@tullio kern[i,j] := 1/(1+i^2+j^2) (i in -3:3, j in -3:3)

@tullio out[x,y,c] := begin
    xi = mod(x+i, axes(mat,1)) #xi = ... significa que no será sumado

    @inbounds trunc(Int, mat[xi, mod(y+j),c] * kern[i,j])
end (x in 1:10, y in 1:10)

10×10×1 Array{Int64, 3}:
[:, :, 1] =
 33   50  33  16   9  0  0  0   9  16
 50  101  50  20  10  0  0  0  10  20
 33   50  33  16   9  0  0  0   9  16
 16   20  16  11   7  0  0  0   7  11
  9   10   9   7   5  0  0  0   5   7
  0    0   0   0   0  0  0  0   0   0
  0    0   0   0   0  0  0  0   0   0
  0    0   0   0   0  0  0  0   0   0
  9   10   9   7   5  0  0  0   5   7
 16   20  16  11   7  0  0  0   7  12

In [65]:
offsets = [(a,b) for a in -2:2 for b in -2:2 if a>=b]

@tullio out[x,y,1] = begin
    a,b = offsets[k]
    i = clamp(x+a, extrema(axes(mat,1))...)
    @inbounds mat[i, clamp(y+b, extrema(axes(mat,2))...),1] * 10
end

10×10×1 Array{Int64, 3}:
[:, :, 1] =
 1010  1010  1010  1010  0  0  0   0   0   0
    0  1010  1010  1010  0  0  0   0   0   0
    0     0  1010  1010  0  0  0   0   0   0
    0     0     0  1010  0  0  0   0   0   0
    0     0     0     0  0  0  0   0   0   0
    0     0     0     0  0  0  0   0   0   0
    0     0     0     0  0  0  0   0   0   0
    0     0     0     0  0  0  0  10  20  30
    0     0     0     0  0  0  0  10  30  50
    0     0     0     0  0  0  0  10  30  60

## Opciones en Tullio

La configuración por defecto es `@tullio threads=true fastmath=true avx=true cuda=256 grad=Base verbose=false`.

Cada opción hace lo siguiente:

- `threads=false` apaga el uso de hilos, mientras que `threads=64^3` es un límite en el cual se divide el trabajo (reemplazando la inferencia hecha por la macro).

- `avx=false` apaga el uso de `LoopVectorization`, mientras que `avx=4` inserta `@avx unroll=4 for i in ...`

- `grad=false` apaga el cálculo de gradiente; `grad=Dual` lo cambia para usar `ForwardDiff` (el cual debe estar cargado).

- `nograd=A` apaga el cálculo de gradiente solo para `A`. Si se quisiera apagar para más elementos, se indican como una tupla (ex. (A, B, C)).

- `tensor=false` apaga el uso de `TensorOperations`.

- `verbose=true`imprime cosas como los rangos de índices inferidos, cálculos de gradients. `verbose=2` imprime **todo**.

- `A[i,j] := ... ` crea un nuevo arreglo, mientras que `A[i,j] = ...` y `A[i,j] +=...` escriben sobre un arreglo existente.

- `init=0.0` indica el valor inicial para reducciones. Para +, *, min, max, &, | tiene valores por defecto sensibles. Para otras reducciones, utiliza cero.

### Opciones Implícitas

- Índices sin desplazamientos tienen el mismo rango en donde aparezcan, pero aquellos con desplazamientos corren en la intersección de los posibles rangos.

- Los índices desplazados de salida deben empezar en 1, a menos que se encuentre el módulo `OffsetArrays` .

- El uso de `@avx` y el cálculo de gradientes se desactiva con sintaxis suficientemente compleja (como arreglos de arreglos).

- Los gradientes sólo están definidos con reducciones sobre (+), min y max.

- Solo se construyen Kernels de GPU cuando tanto `KernelAbstractions` y `CUDA` son visibles. Por defecto se envía `cuda=256` como `kernel(CUDA(), 256)`.

- Los kernels de CPU de `KernelAbstractions` sólo se llaman cuando `thrads=false`. Actualmente no son muy rápidos, pero puede ser usado como pruebas si es necesario.

### Referencias

Para más referencias, lea la [documentación oficial](https://github.com/mcabbott/Tullio.jl/blob/master/README.md) en GitHub o en [juliaHub](https://docs.juliahub.com/Tullio/PIgzC/0.1.1/autodocs/).